In [13]:
import os
from pathlib import Path

path = str(Path(os.path.abspath(os.getcwd())).absolute())

savePath =  path + '\\graph\\'

dataPath = path + '\\data\\padova-mestre-veneto.osm.pbf'

In [14]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace, FOAF

from rdflib.namespace import XSD
OSMO = Namespace("http://www.dei.unipd.it/database2/cutomOSMOntology#")

In [15]:
from pyrosm import OSM

osm = OSM(dataPath)

In [16]:
nodes, ways = osm.get_network(network_type='all', nodes=True)

In [164]:
nodes.head()

,lon,lat,tags,timestamp,version,changeset,id,geometry
0,12.349529,45.451422,None,0,0,0,27181827,POINT (12.34953 45.45142)
1,12.350457,45.452304,None,0,0,0,5591132061,POINT (12.35046 45.45230)
2,12.350471,45.452319,None,0,0,0,1830203504,POINT (12.35047 45.45232)
3,12.350487,45.452335,None,0,0,0,5591132059,POINT (12.35049 45.45234)
4,12.351324,45.453154,None,0,0,0,1854858007,POINT (12.35132 45.45315)


# Add nodes to graph

In [165]:
g = Graph()

g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("osmo", OSMO)

In [166]:
%%time
#measure execution time

for index, row in nodes.iterrows():
    # Create the node to add  to the Graph
    # the node has the namespace + the movie id as URI
    Node = URIRef(OSMO["node"+str(row['id'])])
    g.add((Node, RDF.type, OSMO.Node))
    g.add((Node, OSMO['hasId'], Literal(row['id'], datatype=XSD.integer)))
    g.add((Node, OSMO['hasLatitude'], Literal(row['lat'], datatype=XSD.decimal)))
    g.add((Node, OSMO['hasLongitude'], Literal(row['lon'], datatype=XSD.decimal)))

CPU times: total: 1min 20s
Wall time: 1min 22s


# Write nodes

In [167]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'nodes.ttl', 'wb') as file:
    file.write(g.serialize(format='turtle', encoding='utf-8'))

--- saving serialization ---
CPU times: total: 1min 10s
Wall time: 1min 10s


In [168]:
ways.head(100)

,access,area,bicycle,bicycle_road,bridge,busway,cycleway,est_width,foot,footway,...,width,id,timestamp,version,tags,osm_type,geometry,u,v,length
0,None,None,None,None,None,None,None,None,None,None,...,None,4429697,0,-1,"{""visible_name"":""FONDAMENTA VETRAI;FONDAMENTA ...",way,"LINESTRING (12.34953 45.45142, 12.35046 45.45230)",27181827,5591132061,121.958
1,None,None,None,None,None,None,None,None,None,None,...,None,4429697,0,-1,"{""visible_name"":""FONDAMENTA VETRAI;FONDAMENTA ...",way,"LINESTRING (12.35046 45.45230, 12.35047 45.45232)",5591132061,1830203504,1.997
2,None,None,None,None,None,None,None,None,None,None,...,None,4429697,0,-1,"{""visible_name"":""FONDAMENTA VETRAI;FONDAMENTA ...",way,"LINESTRING (12.35047 45.45232, 12.35049 45.45234)",1830203504,5591132059,2.187
3,None,None,None,None,None,None,None,None,None,None,...,None,4429697,0,-1,"{""visible_name"":""FONDAMENTA VETRAI;FONDAMENTA ...",way,"LINESTRING (12.35049 45.45234, 12.35132 45.45315)",5591132059,1854858007,111.968
4,None,None,None,None,None,None,None,None,None,None,...,None,4429697,0,-1,"{""visible_name"":""FONDAMENTA VETRAI;FONDAMENTA ...",way,"LINESTRING (12.35132 45.45315, 12.35175 45.45356)",1854858007,5591092515,56.281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,None,None,None,None,None,None,None,None,None,...,None,4710518,0,-1,None,way,"LINESTRING (12.27602 45.50525, 12.27599 45.50434)",4271309298,4271309044,100.993
96,None,None,None,None,None,None,None,None,None,None,...,None,4710518,0,-1,None,way,"LINESTRING (12.27599 45.50434, 12.27599 45.50426)",4271309044,29985893,9.654
97,None,None,None,None,None,None,None,None,None,None,...,None,4710518,0,-1,None,way,"LINESTRING (12.27599 45.50426, 12.27598 45.50405)",29985893,29985918,23.133
98,None,None,None,None,None,None,None,None,None,None,...,None,4710519,0,-1,None,way,"LINESTRING (12.27435 45.50533, 12.27447 45.50533)",29985899,4270136411,9.276


# Add roads to graph
## First group roads by id and aggregate lengths and nodes

In [169]:
ways_grouped = ways.groupby('id').agg({'name':'first',
                        'maxspeed':'first',
                        'lanes':'first',
                        'lit':'first',
                        'oneway':'first',
                        'surface':'first',
                        'overtaking':'first',
                        'bicycle_road':'first',
                        'footway':'first',
                        'motorroad':'first',
                        'busway':'first',
                             'length': sum,
                             'u': list,
                             'v': 'last' }).reset_index()

## Initialize graph and insert roads

In [170]:
g = Graph()
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("osmo", OSMO)

In [171]:
%%time
#measure execution time

for index, row in ways_grouped.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Road = URIRef(OSMO["road"+str(row['id'])])

    for uindex, nodeId in enumerate(row['u']):
        Node = URIRef(OSMO["node"+str(nodeId)])

        if uindex == 0:
            g.add((Road, OSMO['hasStartNode'], Node))
        else:
            g.add((Road, OSMO['hasMiddleNode'], Node))

    endNode = URIRef(OSMO["node"+str(row['v'])])
    g.add((Road, OSMO['hasEndNode'], endNode))
    g.add((Road, RDF.type, OSMO.Road))
    g.add((Road, OSMO['hasId'], Literal(row['id'], datatype=XSD.integer)))
    g.add((Road, OSMO['hasName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Road, OSMO['hasLength'], Literal(round(ways[ways['id'] == row['id']]['length'].sum(), 2), datatype=XSD.decimal)))
    g.add((Road, OSMO['hasMaxSpeed'], Literal(row['maxspeed'], datatype=XSD.integer)))
    g.add((Road, OSMO['hasNumberOfLanes'], Literal(0 if row['lanes'] is None else row['lanes'], datatype=XSD.integer)))
    g.add((Road, OSMO['hasSurface'], Literal(row['surface'], datatype=XSD.string)))
    g.add((Road, OSMO['isLit'], Literal(True if row['lit'] == 'yes' else False, datatype=XSD.boolean)))
    g.add((Road, OSMO['isOneWay'], Literal(True if row['oneway'] == 'yes' else False, datatype=XSD.boolean)))
    g.add((Road, OSMO['isOvertakingAllowed'], Literal(True if row['overtaking'] == 'yes' else False, datatype=XSD.boolean)))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#integer, Converter=<class 'int'>
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\envs\boring-wozniak\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: invalid literal for int() with base 10: 'IT:urban'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#integer, Converter=<class 'int'>
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\envs\boring-wozniak\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: invalid literal for int() with base 10: '1.5'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#integer, Converter=<class 'int'>
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\envs\boring-wo

CPU times: total: 3min 5s
Wall time: 3min 7s


# Write roads

In [172]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'roads.ttl', 'wb') as file:
    file.write(g.serialize(format='turtle', encoding='utf-8'))

--- saving serialization ---
CPU times: total: 1min 11s
Wall time: 1min 13s
